In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
train_filenames = [
    "../mydataset/FPS1024/test/FPS1024_0_0.tfrecords",
]
tr_dataset = tf.data.TFRecordDataset(train_filenames).shuffle(1000000)

In [ ]:
def decode(serialized_example, total_num_point):
    features = tf.io.parse_example(
        [serialized_example],
        features={
            "xyz": tf.io.FixedLenFeature([total_num_point, 3], tf.float32),
            "rgb": tf.io.FixedLenFeature([total_num_point, 3], tf.float32),
            "translation": tf.io.FixedLenFeature([3], tf.float32),
            "quaternion": tf.io.FixedLenFeature([4], tf.float32),
            "num_valid_points_in_segment": tf.io.FixedLenFeature([], tf.int64),
            "seq_id": tf.io.FixedLenFeature([], tf.int64),
            "frame_id": tf.io.FixedLenFeature([], tf.int64),
            "class_id": tf.io.FixedLenFeature([], tf.int64),
        },
    )
    return features


BATCH_SIZE = 1
TOTAL_NUM_POINT = 1024

tr_dataset = tr_dataset.map(lambda x: decode(x, TOTAL_NUM_POINT))

In [ ]:
print(tr_dataset)

In [ ]:
element = iter(tr_dataset).get_next()

In [ ]:
# seg = tf.concat(values=[element.get("xyz"), element.get("rgb")], axis=0)
seg = tf.squeeze(element.get("xyz"))
seg.shape

In [ ]:
rotation_matrix = tfgt.rotation_matrix_3d.from_quaternion(
    element.get("quaternion"),
)

In [ ]:
# Flatten tensor to 1D array
pos = tf.concat(
    [
        tf.reshape(rotation_matrix, -1),
        tf.reshape(element.get("translation"), -1),
        tf.constant([0, 0, 0, 1], dtype=tf.float32),
        tf.reshape(tf.cast(element.get("class_id"), tf.float32), -1),
    ],
    axis=0,
)

In [ ]:
np.save("../mydataset/my_train/0000_pos.npy", pos.numpy())

In [ ]:
np.save("../mydataset/my_train/0000_seg.npy", seg.numpy())

## Read TFRecord in PyTorch

In [ ]:
import os

import torch
from tfrecord.tools import tfrecord2idx
from tfrecord.torch.dataset import TFRecordDataset

DIR = "../mydataset/FPS1024/test"
FILE_NAME = "FPS1024_0_0.tfrecords"
FILE_PATH = os.path.join(DIR, FILE_NAME)
index_path = FILE_PATH.replace("tfrecords", "idx")
description = {
    "xyz": "float",
    "rgb": "float",
    "translation": "float",
    "quaternion": "float",
    "num_valid_points_in_segment": "int",
    "seq_id": "int",
    "frame_id": "int",
    "class_id": "int",
}

dataset = TFRecordDataset(FILE_PATH, index_path, description)
loader = torch.utils.data.DataLoader(dataset, batch_size=32)

data = next(iter(loader))

In [ ]:
from pytorch3d import transforms

transforms.quaternion_to_matrix(data["quaternion"])

In [ ]:
transforms.quaternion_to_axis_angle(data["quaternion"])

In [ ]:
import cv2

rotation_matrix = transforms.quaternion_to_matrix(data["quaternion"])
rotation_matrix = rotation_matrix[0, :, :]
axag = cv2.Rodrigues(rotation_matrix.numpy())[0].flatten()
axag

In [ ]:
transforms.quaternion_to_axis_angle(data["quaternion"])

### Split `.tfrecord` files into train, test, and val dataset

In [ ]:
import glob

ROOT_DIR = os.path.dirname(os.getcwd())
DIR_NAME = "mydataset/FPS1024"

FILE_PATHS = list(glob.iglob(os.path.join(ROOT_DIR, DIR_NAME, "*.tfrecords")))

In [ ]:
assert len(FILE_PATHS) == 42

In [ ]:
# Each tfrecord files have ~45000 elements, 50000 is enough for randomnization
BUFFER_SIZE = 50000
for f in FILE_PATHS:
    dataset = tf.data.TFRecordDataset(f).shuffle(buffer_size=BUFFER_SIZE)

    ds_size = len(list(dataset))
    train_size, test_size, val_size = [int(ds_size * s) for s in [0.8, 0.1, 0.1]]

    train_dataset = dataset.take(train_size)
    test_dataset = dataset.skip(train_size)
    val_dataset = test_dataset.skip(test_size)
    test_dataset = test_dataset.take(test_size)

    dir_path = os.path.join(ROOT_DIR, DIR_NAME)
    for d, dataset in [
        ("train", train_dataset),
        ("test", test_dataset),
        ("val", val_dataset),
    ]:
        file_path = os.path.join(dir_path, d, os.path.basename(f)[12:])
        print("Saving", file_path)
        writer = tf.data.experimental.TFRecordWriter(file_path)
        writer.write(dataset)

### Create Index Path for all `.tfrecords` files 

In [ ]:
import itertools

FILE_PATHS = {
    d: list(glob.iglob(os.path.join(ROOT_DIR, DIR_NAME, d, "*.tfrecords")))
    for d in ["train", "test", "val"]
}

In [ ]:
assert len(FILE_PATHS) == 3
assert len(FILE_PATHS.get("train")) == 42
assert len(FILE_PATHS.get("test")) == 42
assert len(FILE_PATHS.get("val")) == 42

`.idx` file is required for multi-file loading

In [ ]:
for paths in FILE_PATHS.values():
    for path in paths:
        idx_path = path.replace("tfrecords", "idx")
        tfrecord2idx.create_index(path, idx_path)

In [ ]:
INDEX_PATHS = {
    d: list(glob.iglob(os.path.join(ROOT_DIR, DIR_NAME, d, "*.idx")))
    for d in ["train", "test", "val"]
}

Make sure we have created `.idx` files for all `.tfrecord` files

In [ ]:
assert len(INDEX_PATHS) == 3
assert len(INDEX_PATHS.get("train")) == 42
assert len(INDEX_PATHS.get("test")) == 42
assert len(INDEX_PATHS.get("val")) == 42

Save those paths to a JSON file for later use

In [ ]:
import json
import pathlib

get_filename = lambda x: pathlib.Path(x).stem

filenames = map(get_filename, FILE_PATHS.get("train"))
# Convert map object to dict
filenames_dict = {filename: 1 for filename in filenames}

# Save to json
json_path = os.path.join(ROOT_DIR, DIR_NAME, f"FPS1024.json")

with open(json_path, "w") as f:
    f.write(json.dumps(filenames_dict))

### Now, let's load `.tfrecords` files at once

In [ ]:
from tfrecord.torch.dataset import MultiTFRecordDataset

tfrecord_pattern = os.path.join(ROOT_DIR, DIR_NAME, "train/{}.tfrecords")
index_pattern = os.path.join(ROOT_DIR, DIR_NAME, "train/{}.idx")
splits = json.load(open(json_path, "r"))


dataset = MultiTFRecordDataset(
    data_pattern=tfrecord_pattern,
    index_pattern=index_pattern,
    splits=splits,
    description=description,
)

loader = torch.utils.data.DataLoader(dataset, batch_size=32)

data = next(iter(loader))
print(data)

In [ ]:
import os
import sys

import numpy as np


class myDatasetConfig(object):
    def __init__(self):
        self.num_class = 21

        # Class name and id map
        # see: https://github.com/GeeeG/CloudPose/blob/d6410dc4af9a58c00511e34fdc41c6cfd9f96cba/ycb_video_data_tfRecords/script/2_dataset_to_tfRecord_small.py
        self.class_id = {
            "master chef can": 0,
            "cracker box": 1,
            "suger box": 2,
            "tomato soup can": 3,
            "mustard bottle": 4,
            "tuna fish can": 5,
            "pudding box": 6,
            "gelatin box": 7,
            "potted meat can": 8,
            "banana": 9,
            "pitcher base": 10,
            "bleach cleanser": 11,
            "bowl": 12,
            "mug": 13,
            "drill": 14,
            "wood block": 15,
            "scissors": 16,
            "large marker": 17,
            "large clapm": 18,
            "extra large clamp": 19,
            "foam brick": 20,
        }

        self.id_class = {self.class_id[t]: t for t in self.class_id}

        # 2D array
        self.onehot_encoding = np.eye(self.num_class)[
            np.array([range(self.num_class)]).reshape(-1)
        ]

    def sem2class(self, cls):
        # Select ith row of the 2D array
        onehot = self.onehot_encoding[int(cls), :]
        return onehot

    def size2class(self, class_name):
        """Convert 3D box size (l,w,h) to size class and size residual"""
        size_class = self.class_id[class_name]  # 0
        # size_residual = size - self.type_mean_size[type_name]  # 尺寸
        return size_class

    def class2size(self, pred_cls):
        """Inverse function to size2class"""
        mean_size = self.type_mean_size[self.id_class[pred_cls]]
        return mean_size

    def class2sem(self, pred_cls):
        """Given point_cloud_with_cls, return class name"""
        class_id = np.argwhere(pred_cls[0, -self.num_class :] == 1).flatten()[0]
        return class_id

    def id2class(self, cls):
        """Return class name given class id."""
        return self.id_class[cls]

In [ ]:
DC = myDatasetConfig()

In [ ]:
point_cloud = np.random.rand(1024, 3)
one_hot = DC.sem2class(1)
one_hot_ex_rep = np.repeat(np.expand_dims(one_hot, axis=0), 1024, axis=0)
point_cloud_with_cls = np.concatenate((point_cloud, one_hot_ex_rep), axis=1)

In [ ]:
np.argwhere(point_cloud_with_cls[0, -21:] == 1).flatten()[0]

In [ ]:
DC.class2sem(point_cloud_with_cls)

In [ ]:
DC.id2class(2)

In [ ]:
point_cloud_with_cls[:, :3]

In [ ]:
from pyntcloud import PyntCloud

In [ ]:
mes = PyntCloud.from_file("../myDataset/ycb_video_obj_ply/002_master_chef_can.ply")

In [ ]:
mesh.plot(
    return_scene=True,
)

In [ ]:
import numpy as np
import pandas as pd

point_cloud = pd.DataFrame(np.random.rand(1024, 3), columns=["x", "y", "z"])
cloud = PyntCloud(point_cloud)

In [ ]:
cloud.plot()